## IBM Data Science Course Capstone Project




by A.Rakhmetov






### Assignment Week 1

In [2]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Assignment Week 3. Segmenting and Clustering Neighborhoods in Toronto

#### a) Parsing the Toronto postal codes data from Wikipedia site:

In [3]:
from bs4 import BeautifulSoup
import requests


wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_page,'html.parser')

#The html table with postal codes:
My_table = soup.find('table', class_='wikitable sortable')


#### b) Preparing the 2D NumPy array for the dataframe operations in Pandas:

In [4]:

List=[]
for cell in My_table.find_all('td'):
    List.append(cell.text.replace('\n',''))
   
data = np.reshape(np.array(List), (-1,3))

print(data.shape)
data[0:5]

(289, 3)


array([['M1A', 'Not assigned', 'Not assigned'],
       ['M2A', 'Not assigned', 'Not assigned'],
       ['M3A', 'North York', 'Parkwoods'],
       ['M4A', 'North York', 'Victoria Village'],
       ['M5A', 'Downtown Toronto', 'Harbourfront']], dtype='<U49')

#### c) Implementing and pre-processing the dataframe according to the requirements:

In [5]:
# Creating the dataframe out of the numpy array:
df = pd.DataFrame({'PostalCode':data[:,0], 'Borough':data[:,1], 'Neighborhood':data[:,2]})

#Getting rid of "Not assigned" cells in the 'Borough' and 'Neighborhood' columns:
df = df[df.Borough != "Not assigned"]
df['Neighborhood'] = np.where(df['Neighborhood']=='Not assigned', df['Borough'], df['Neighborhood'])

# Grouping the Neighborhoods in lists according to the postal codes:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list).reset_index()

# Merging the lists into one string separated with commas:
df['Neighborhood'] = df['Neighborhood'].str.join(', ')


df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df.shape

(103, 3)

#### d) Getting the geospatial coordinates of each Postal Code:

In [5]:
# Installing the geocoder:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0       conda-forge
    orderedset: 2.0-py36_0        conda-forge
    ratelim:    0.1.6-py36_0      conda-forge

The following packages will be UPDATED:

    openssl:    1.0.2p-h470a237_1 conda-forge --

In [12]:
import geocoder 

g = geocoder.google("M5G, Toronto, Ontario")
print(g.latlng)


None


Unfortunarely, after numerous fail attempts to obtain geospatial coordinates from the Geocoder API, I had to use the prepared csv file 

In [34]:
gdf=pd.read_csv('http://cocl.us/geospatial_data')
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# Merging the dataframes in one:

toronto_df = pd.merge(df,gdf, left_on = 'PostalCode', right_on = 'Postal Code').drop('Postal Code', 1)

toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [36]:
toronto_df.shape

(103, 5)